In [ ]:
!pip install ../input/torchlibrosa/torchlibrosa-0.0.5-py3-none-any.whl > /dev/null

In [ ]:
import cv2
import audioread
import logging
import gc
import os
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import random
import time
import warnings

import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torchdata

from contextlib import contextmanager
from joblib import Parallel, delayed
from pathlib import Path
from typing import Optional
from sklearn.model_selection import StratifiedKFold, GroupKFold

from albumentations.core.transforms_interface import ImageOnlyTransform
from torchlibrosa.stft import LogmelFilterBank, Spectrogram
from torchlibrosa.augmentation import SpecAugmentation
from tqdm import tqdm

import albumentations as A
import albumentations.pytorch.transforms as T

import matplotlib.pyplot as plt

In [ ]:
SR = 32000
USE_SEC = 20


def Audio_to_Array(path):
    y, sr = sf.read(path, always_2d=True)
    y = np.mean(y, 1) # there is (X, 2) array
    if len(y) > SR:
        y = y[SR:-SR]
    if len(y) < SR * USE_SEC:
        return [y, y]
    chunks = int(len(y)//(SR*USE_SEC))
    Ys = []
    for i in range(chunks):
        Ys.append(y[i*SR*USE_SEC : (i+1)*SR*USE_SEC])
    Ys.append(y[-SR*USE_SEC :])         
    return Ys


AUDIO_PATH = '../input/birdclef-2022/train_audio'

train = pd.read_csv('../input/birdies/train_H.csv')
train["file_path"] = AUDIO_PATH + '/' + train['filename']
paths = train["file_path"].values

In [ ]:
NUM_WORKERS = 4
CLASSES = ["akiapo", "aniani", "apapan", "barpet", "crehon", "elepai", "ercfra", "hawama", 
           "hawcre", "hawgoo", "hawhaw", "hawpet1", "houfin", "iiwi", "jabwar", "maupar", 
           "omao", "puaioh", "skylar", "warwhe1", "yefcan"]
#print(CLASSES)
lens = []

for dir_ in CLASSES:
    #print(dir_)
    _ = os.makedirs('train_np/' + dir_, exist_ok=True)
    
for apath in paths:
    Yy = Audio_to_Array(apath)
    chs = len(Yy)
    #print(apath, '....', chs)
    lens.append(chs)
    for i in range(chs):
        save_path = "train_np/" + "/".join(apath.split('/')[-2:])+'_'+str(i+1)
        np.save(save_path, Yy[i])        
    

In [ ]:
train['chunks'] = lens
train.to_csv('train_chunks.csv', index=False)